---

_You are currently looking at **version 1.0** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-machine-learning/resources/bANLa) course resource._

---

## Assignment 4 - Understanding and Predicting Property Maintenance Fines

This assignment is based on a data challenge from the Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)). 

The Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)) and the Michigan Student Symposium for Interdisciplinary Statistical Sciences ([MSSISS](https://sites.lsa.umich.edu/mssiss/)) have partnered with the City of Detroit to help solve one of the most pressing problems facing Detroit - blight. [Blight violations](http://www.detroitmi.gov/How-Do-I/Report/Blight-Complaint-FAQs) are issued by the city to individuals who allow their properties to remain in a deteriorated condition. Every year, the city of Detroit issues millions of dollars in fines to residents and every year, many of these fines remain unpaid. Enforcing unpaid blight fines is a costly and tedious process, so the city wants to know: how can we increase blight ticket compliance?

The first step in answering this question is understanding when and why a resident might fail to comply with a blight ticket. This is where predictive modeling comes in. For this assignment, your task is to predict whether a given blight ticket will be paid on time.

All data for this assignment has been provided to us through the [Detroit Open Data Portal](https://data.detroitmi.gov/). **Only the data already included in your Coursera directory can be used for training the model for this assignment.** Nonetheless, we encourage you to look into data from other Detroit datasets to help inform feature creation and model selection. We recommend taking a look at the following related datasets:

* [Building Permits](https://data.detroitmi.gov/Property-Parcels/Building-Permits/xw2a-a7tf)
* [Trades Permits](https://data.detroitmi.gov/Property-Parcels/Trades-Permits/635b-dsgv)
* [Improve Detroit: Submitted Issues](https://data.detroitmi.gov/Government/Improve-Detroit-Submitted-Issues/fwz3-w3yn)
* [DPD: Citizen Complaints](https://data.detroitmi.gov/Public-Safety/DPD-Citizen-Complaints-2016/kahe-efs3)
* [Parcel Map](https://data.detroitmi.gov/Property-Parcels/Parcel-Map/fxkw-udwf)

___

We provide you with two data files for use in training and validating your models: train.csv and test.csv. Each row in these two files corresponds to a single blight ticket, and includes information about when, why, and to whom each ticket was issued. The target variable is compliance, which is True if the ticket was paid early, on time, or within one month of the hearing data, False if the ticket was paid after the hearing date or not at all, and Null if the violator was found not responsible. Compliance, as well as a handful of other variables that will not be available at test-time, are only included in train.csv.

Note: All tickets where the violators were found not responsible are not considered during evaluation. They are included in the training set as an additional source of data for visualization, and to enable unsupervised and semi-supervised approaches. However, they are not included in the test set.

<br>

**File descriptions** (Use only this data for training your model!)

    train.csv - the training set (all tickets issued 2004-2011)
    test.csv - the test set (all tickets issued 2012-2016)
    addresses.csv & latlons.csv - mapping from ticket id to addresses, and from addresses to lat/lon coordinates. 
     Note: misspelled addresses may be incorrectly geolocated.

<br>

**Data fields**

train.csv & test.csv

    ticket_id - unique identifier for tickets
    agency_name - Agency that issued the ticket
    inspector_name - Name of inspector that issued the ticket
    violator_name - Name of the person/organization that the ticket was issued to
    violation_street_number, violation_street_name, violation_zip_code - Address where the violation occurred
    mailing_address_str_number, mailing_address_str_name, city, state, zip_code, non_us_str_code, country - Mailing address of the violator
    ticket_issued_date - Date and time the ticket was issued
    hearing_date - Date and time the violator's hearing was scheduled
    violation_code, violation_description - Type of violation
    disposition - Judgment and judgement type
    fine_amount - Violation fine amount, excluding fees
    admin_fee - $20 fee assigned to responsible judgments
state_fee - $10 fee assigned to responsible judgments
    late_fee - 10% fee assigned to responsible judgments
    discount_amount - discount applied, if any
    clean_up_cost - DPW clean-up or graffiti removal cost
    judgment_amount - Sum of all fines and fees
    grafitti_status - Flag for graffiti violations
    
train.csv only

    payment_amount - Amount paid, if any
    payment_date - Date payment was made, if it was received
    payment_status - Current payment status as of Feb 1 2017
    balance_due - Fines and fees still owed
    collection_status - Flag for payments in collections
    compliance [target variable for prediction] 
     Null = Not responsible
     0 = Responsible, non-compliant
     1 = Responsible, compliant
    compliance_detail - More information on why each ticket was marked compliant or non-compliant


___

## Evaluation

Your predictions will be given as the probability that the corresponding blight ticket will be paid on time.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC). 

Your grade will be based on the AUC score computed for your classifier. A model which with an AUROC of 0.7 passes this assignment, over 0.75 will recieve full points.
___

For this assignment, create a function that trains a model to predict blight ticket compliance in Detroit using `train.csv`. Using this model, return a series of length 61001 with the data being the probability that each corresponding ticket from `test.csv` will be paid, and the index being the ticket_id.

Example:

    ticket_id
       284932    0.531842
       285362    0.401958
       285361    0.105928
       285338    0.018572
                 ...
       376499    0.208567
       376500    0.818759
       369851    0.018528
       Name: compliance, dtype: float32

In [176]:
import pandas as pd
import numpy as np

def blight_model():
    
    # Your code here
    
    return # Your answer here



# read training data
train_df = pd.read_csv('train.csv', encoding='ISO-8859-1')

# analyze features
# train_df['compliance'].unique()
# train_df.describe(include='all')
# train_df.corr()

train_df = train_df[~train_df['compliance'].isna()]
train_df.set_index('ticket_id', inplace=True)

# read location lookups
addresses_df = pd.read_csv('addresses.csv')
latlons_df = pd.read_csv('latlons.csv')
loc_info_df = addresses_df.merge(latlons_df, on='address').drop('address', axis=1)
loc_info_df.set_index('ticket_id', inplace=True)


test_df = pd.read_csv('test.csv')
test_df2 = test_df.merge(loc_info_df, on='ticket_id')
test_df2.set_index('ticket_id', inplace=True)
#test_df2


In [177]:
import time
from sklearn.preprocessing import MinMaxScaler
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import roc_auc_score

# drop additional columns except compliance
drop_cols = train_df.columns.difference(test_df.columns).drop('compliance')
train_df2 = train_df.drop(drop_cols, axis=1)
#train_df2 = train_df2.merge(loc_info_df, on='ticket_id')
train_df2 = train_df2.join(loc_info_df)

""" 
# drop object columns - errors when using encoder later
typ = train_df2.dtypes
str_cols = typ[typ=='object'].index.tolist()
train_df2 = train_df.drop(str_cols, axis=1)

# Fill nan values before using Encoder
# train_df2[[col for col in train_df2 if train_df2.isna().any()[col] ]].dtypes
train_df2['violation_zip_code'].fillna(0, inplace=True)
train_df2['mailing_address_str_number'].fillna(0, inplace=True)
train_df2['lat'].fillna(train_df2['lat'].mean(), inplace=True)
train_df2['lon'].fillna(train_df2['lon'].mean(), inplace=True)
 """

# create X and y for training set
y = train_df2['compliance']
train_df3 = train_df2.drop('compliance', axis=1)

# columns to be scaled
scale_cols = ['violation_street_number', 'fine_amount', 'admin_fee', 'state_fee','late_fee', 'discount_amount', 'clean_up_cost', 'judgment_amount']

""" 
col_xfm = ColumnTransformer(remainder='passthrough', 
                            transformers=[('minmax', MinMaxScaler(), scale_cols)])
xfm = col_xfm.fit_transform(train_df3)
 """

# create a validation set to check model performance
X_train, X_val, y_train, y_val = train_test_split(train_df3, y, random_state=0)

# Encode object columns
typ = X_train.dtypes
str_cols = typ[typ=='object'].index.tolist()


from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
X_train2 = X_train.copy()
X_val2 = X_val.copy()
for c in str_cols:
    print(f'Col={c}')
    if 'zip' in c:
        print('replacing values in zip')
        X_train2[c] = X_train2[c].str.slice(0, 5)
        X_train2[c] = X_train2[c].str.replace('[^\d]', '', regex=True)
        X_train2[c].fillna(0, inplace=True)
        X_val2[c].fillna(0, inplace=True)
        #X_train2[c] = X_train2[X_train2[c].astype('str').str.contains('[^\d]', regex=True)][c].str.replace('[^\d]', '', regex=True)

    if 'date' in c:
        X_train2[c] = pd.to_datetime(X_train2[c])  
        X_val2[c] = pd.to_datetime(X_val2[c])  
    else:
        X_train2[c] = pd.to_numeric(X_train2[c], errors='ignore', downcast='float')
        X_val2[c] = pd.to_numeric(X_val2[c], errors='ignore', downcast='float')
    #if X_train2[c].isnull().all():
    #    print(f"drop={c}")
    #    X_train2.drop(c, inplace=True)
    #else:
    print(f'Initial Dtype={X_train2[c].dtype}')
    X_train2[c].fillna('000', inplace=True)
    X_val2[c].fillna('000', inplace=True)
    X_train2[c] = encoder.fit_transform(X_train2[c])
    X_val2[c] = encoder.fit_transform(X_val2[c])
    print(f'Final Dtype={X_train2[c].dtype}')




Col=agency_name
Initial Dtype=object
Final Dtype=int64
Col=inspector_name
Initial Dtype=object
Final Dtype=int64
Col=violator_name
Initial Dtype=object
Final Dtype=int64
Col=violation_street_name
Initial Dtype=object
Final Dtype=int64
Col=mailing_address_str_name
Initial Dtype=object
Final Dtype=int64
Col=city
Initial Dtype=object
Final Dtype=int64
Col=state
Initial Dtype=object
Final Dtype=int64
Col=zip_code
replacing values in zip
Initial Dtype=float32


TypeError: argument must be a string or number

In [182]:
#pd.to_numeric(X_train2['zip_code'], errors='ignore', downcast='float')
# replace non-numeric zip with nan
#X_train2[X_train2['zip_code'].astype('str').str.contains('[^\d]', flags=re.IGNORECASE, regex=True)]['zip_code'] #.str.replace('[^\d]', '', regex=True)
# replace hyphen in numeric zip
#X_train2[X_train2['zip_code'].str.contains('-', regex=True)==True]['zip_code']  #.str.replace('-.*', '')
print(X_train['zip_code'].unique())
X_train2['zip_code'].unique().tolist()

['19040' '48314' '48206' ... '94134' 10120 '92519']


[19040.0,
 48314.0,
 48206.0,
 0.0,
 48219.0,
 48237.0,
 48116.0,
 48089.0,
 48228.0,
 48211.0,
 48239.0,
 91311.0,
 48205.0,
 48236.0,
 48167.0,
 48221.0,
 48227.0,
 48325.0,
 48034.0,
 48223.0,
 48214.0,
 48329.0,
 77320.0,
 48185.0,
 48210.0,
 28602.0,
 48238.0,
 48111.0,
 48188.0,
 48208.0,
 19101.0,
 48207.0,
 48126.0,
 48075.0,
 92705.0,
 48244.0,
 74135.0,
 21701.0,
 48037.0,
 48202.0,
 48302.0,
 48183.0,
 48224.0,
 48076.0,
 48120.0,
 48213.0,
 48209.0,
 91765.0,
 48310.0,
 48044.0,
 48235.0,
 48174.0,
 48192.0,
 48015.0,
 48203.0,
 48164.0,
 33402.0,
 48204.0,
 48220.0,
 48191.0,
 48312.0,
 32826.0,
 20910.0,
 48127.0,
 48146.0,
 90247.0,
 48128.0,
 93063.0,
 48234.0,
 48336.0,
 48240.0,
 33027.0,
 60126.0,
 29063.0,
 34229.0,
 92630.0,
 48036.0,
 10601.0,
 60047.0,
 93065.0,
 48906.0,
 29054.0,
 48217.0,
 48216.0,
 48225.0,
 19034.0,
 91754.0,
 48331.0,
 48066.0,
 49640.0,
 48322.0,
 8054.0,
 48160.0,
 48175.0,
 92660.0,
 72921.0,
 48047.0,
 48025.0,
 48081.0,
 48317.0,
 4801

In [68]:

# scale features in training set
scaler = MinMaxScaler()
X_train_scaled = X_train2
X_train_scaled[scale_cols] = scaler.fit_transform(X_train2[scale_cols])
X_val_scaled = X_val2
X_val_scaled[scale_cols] = scaler.transform(X_val2[scale_cols])



# try different classifiers
classifiers = [LogisticRegression(), GaussianNB(),
               RandomForestClassifier(n_estimators=100, max_features=10, max_depth=7), 
               MLPClassifier(hidden_layer_sizes=[100, 100], random_state=0,
                             activation='relu', alpha=1.0, solver='adam')]
classifiers



# check scores of each classifier
#for clf in classifiers:
#    start_time = time.time()
#    fitted = clf.fit(X_train_scaled, y_train)
#    y_predict = fitted.predict(X_val_scaled)
#    auc = roc_auc_score(y_val, y_predict)
#    end_time = time.time()
#    print(f'Time taken = {end_time - start_time}, AUC = {auc} for {clf}')


column=agency_name
Col=agency_name, dtype = object
column=inspector_name
Col=inspector_name, dtype = object
column=violator_name
Col=violator_name, dtype = object
column=violation_street_name
Col=violation_street_name, dtype = object
column=mailing_address_str_name
Col=mailing_address_str_name, dtype = object
column=city
Col=city, dtype = object
column=state
Col=state, dtype = object
column=zip_code
Col=zip_code, dtype = object
column=non_us_str_code
Col=non_us_str_code, dtype = object
column=country
Col=country, dtype = object
column=ticket_issued_date
Col=ticket_issued_date, dtype = datetime64[ns]
column=hearing_date
Col=hearing_date, dtype = datetime64[ns]
column=violation_code
Col=violation_code, dtype = object
column=violation_description
Col=violation_description, dtype = object
column=disposition
Col=disposition, dtype = object
column=grafitti_status
Col=grafitti_status, dtype = float32


In [66]:
#print(test_df.dtypes)
#test_df.head()
train_df2[train_df2.zip_code.isna()]
train_df.columns
train_df2['grafitti_status'].unique()
typ = X_train.dtypes
str_cols = typ[typ=='object'].index.tolist()
for c in str_cols:
    colname = ''.join(c)
    print(f'Colname={c} - {id(c)}')
    if X_train2[c].isnull().all():
        print(f'columnnn{c} - {id(c)}')
        print(f'Colname={colname} - {id(c)}')
        print(f'--Drop--{c} - {id(c)}, {colname} - {id(c)}')
    else:
        print('fillna')

Colname=agency_name - 140553489453936
fillna
Colname=inspector_name - 140553262214000
fillna
Colname=violator_name - 140553262214832
fillna
Colname=violation_street_name - 140553033073248
fillna
Colname=mailing_address_str_name - 140552992133248
fillna
Colname=city - 140553020860528
fillna
Colname=state - 140553469239344
fillna
Colname=zip_code - 140553260966704
fillna
Colname=non_us_str_code - 140553261132464
fillna
Colname=country - 140553488762480
fillna
Colname=ticket_issued_date - 140552992136768
fillna
Colname=hearing_date - 140553259132848
fillna
Colname=violation_code - 140553259132144
fillna
Colname=violation_description - 140552992133568
fillna
Colname=disposition - 140553259133872
fillna
Colname=grafitti_status - 140553259131760
columnnngrafitti_status - 140553259131760
Colname=grafitti_status - 140553259131760
--Drop--grafitti_status - 140553259131760, grafitti_status - 140553259131760


In [3]:

# get test data
X_test = test_df

# scale features in test set
#X_test_scaled = scaler.transform(X_test)

# predict test output


# get prediction probablilities



NameError: name 'scaler' is not defined